In [ ]:
import numpy as np

import os 

import pandas as pd
from scipy.interpolate import interp1d
import bilby
import numpy as np
import matplotlib.pylab as plt
import json
import os
import sys
from bilby.gw.source import *
from numba import jit

#sys.path.append('..')

from glow import lenses
from glow import time_domain_c
from glow import freq_domain_c
from astropy import constants as c

from glow import time_domain
from glow import freq_domain
from glow import tools
import glow.physical_units as pu
#lal.swig_redirect_standard_output_error(True)
GMsun8pi=(8*c.G*c.M_sun*np.pi/c.c**3).decompose().value

## Reproduce [Gravelamps (fig 1)](https://arxiv.org/pdf/2112.07012)

NFW: $k_s = \psi_0$

In [ ]:
Cprec = {'no_warnings':1, 'no_errors':1, 'no_gslerrors':1}
#Cprec = {'no_output':1}

p_prec_t={'C_prec':Cprec}
Psi_PM=lenses.Psi_PointLens()
Psi_SIS=lenses.Psi_SIS()
Psi_NFW=lenses.Psi_NFW(p_phys={'psi0':2, 'xs':1})
#{'tmax':1e7, 'Nt' : 10000}
p_opt_F = {'wmin':1e-3, 'wmax':1e4,'N_below_discard':5,\
                   'N_above_discard':5,\
                   'N_keep':1,'eval_mode':'exact','C_prec':Cprec}
plt.figure(figsize= (10,4))
for i,psi in enumerate([Psi_PM,Psi_SIS,Psi_NFW]):

    plt.subplot(1,3,i+1)
    ws=np.linspace(1e-2,1e2,10000)
    name = psi.default_general_params()[0]['name']
    #print(name, 'here')

    for y in [0.1,0.3,1,3]:
        I = time_domain_c.It_SingleIntegral_C(psi, y, p_prec=p_prec_t)
        Fw = freq_domain_c.Fw_FFT_C(I, p_prec=p_opt_F).eval_Fw(ws)
        plt.semilogx(ws,np.abs(Fw), label='y=' + str(y))
        
    plt.legend()
    plt.xlabel('w')
    plt.ylabel('|F(w)|')
    plt.title(name)
plt.tight_layout()
plt.show()

## Reproduce [GLWORIA NFW & CIS waveforms (fig 3)](https://arxiv.org/pdf/2403.13876)
Conversion of parameters, See 26-28 and cross-check with GloW!

$\psi_0 = \kappa, \quad xs = 1$ 

In [ ]:
psi0=[5,3]
y= [0.2,1]
#{'tmax':1e7, 'Nt' : 10000}
p_opt_F = {'wmin':1e-3, 'wmax':1e4,'N_below_discard':5,\
                   'N_above_discard':5,\
                   'N_keep':1,'eval_mode':'exact','C_prec':Cprec}
plt.figure(figsize= (8,6))
for i in range(len(psi0)):
    psi=lenses.Psi_NFW(p_phys={'psi0':psi0[i], 'xs':1})

    plt.subplot(2,2,2*i+1)
    ws=np.linspace(1e-2,1e3,100000)
    #name = psi.default_general_params()[0]['name']
    #print(name, 'here')

    I = time_domain_c.It_SingleIntegral_C(psi, y[i], p_prec=p_prec_t)
    Fw = freq_domain_c.Fw_FFT_C(I, p_prec=p_opt_F).eval_Fw(ws)
    plt.semilogx(ws,np.abs(Fw))


    #plt.legend()
    plt.xlabel('w')
    plt.ylabel('|F(w)|')
    plt.title(' $\mathcal{\kappa} = %.1f, y = %.1f$'%(psi0[i],y[i]))
    plt.subplot(2,2,2*i+2)
    plt.semilogx(ws,np.angle(Fw))
    #plt.legend()
    plt.xlabel('w')
    plt.ylabel('arg(F(w))')
plt.tight_layout()
plt.suptitle('NFW')
plt.show()

In [ ]:
xc=[0.05,0.2]
y= [0.2,1.5]
#{'tmax':1e7, 'Nt' : 10000}
p_opt_F = {'wmin':1e-3, 'wmax':1e4,'N_below_discard':5,\
                   'N_above_discard':5,\
                   'N_keep':1,'eval_mode':'exact','C_prec':Cprec}
plt.figure(figsize= (8,6))
for i in range(len(psi0)):
    psi=lenses.Psi_CIS(p_phys={'xc':xc[i]})

    plt.subplot(2,2,2*i+1)
    ws=np.linspace(1e-2,1e3,100000)
    #name = psi.default_general_params()[0]['name']
    #print(name, 'here')

    I = time_domain_c.It_SingleIntegral_C(psi, y[i], p_prec=p_prec_t)
    Fw = freq_domain_c.Fw_FFT_C(I, p_prec=p_opt_F).eval_Fw(ws)
    plt.semilogx(ws,np.abs(Fw))


    #plt.legend()
    plt.xlabel('w')
    plt.ylabel('|F(w)|')
    plt.title(' $x_c = %.1f, y = %.1f$'%(xc[i],y[i]))
    plt.subplot(2,2,2*i+2)
    plt.semilogx(ws,np.angle(Fw))
    #plt.legend()
    plt.xlabel('w')
    plt.ylabel('arg(F(w))')
plt.tight_layout()
plt.suptitle('CIS')

plt.show()

#  Reproduce Mishra et al

In [ ]:
Cprec = {'no_warnings':1, 'no_errors':1, 'no_gslerrors':1}
#Cprec = {'no_output':1}
MLz=100*1.5
p_prec_t={'C_prec':Cprec}
Psi_PM=lenses.Psi_PointLens()

p_opt_F = {'wmin':1e-3, 'wmax':1e5,'N_below_discard':6,\
                   'N_above_discard':6,\
                   'N_keep':3,'eval_mode':'exact','C_prec':Cprec}
plt.figure(figsize= (10,3))
frequency_array = np.linspace(0,3e3,10000)
for i,psi in enumerate([Psi_PM]):
    for y in [0.01,0.1,0.3,1.]:
        
         
        p_prec_t.update({'tmax':1e1, 'tmin':1e-5,'Nt':int(1e5)})
        I=time_domain_c.It_SingleIntegral_C(psi, y, p_prec=p_prec_t)
        Fw = freq_domain_c.Fw_FFT_C(I, p_prec=p_opt_F)
        ws=GMsun8pi*MLz*frequency_array
        fw= Fw.eval_Fw(ws)
        plt.subplot(131)

        plt.plot(I.t_grid, I.It_grid/2/np.pi,label='y = '+str(y))
        plt.xlabel(r'$\tau$')
        plt.ylabel(r'$I(\tau)/2\pi$')
    
        plt.xscale('log')
        #plt.xlim(1e-6,1e1)
        #plt.ylim(0.9,4)
        plt.legend()
    
        plt.subplot(132)

        plt.loglog(frequency_array,np.abs(fw),label='y = '+str(y))

        plt.legend()
        plt.xlabel('Frequency')
        plt.ylabel('|F(w)|')
        plt.grid()
        plt.xlim(1,3e3)

        plt.subplot(133)

        plt.semilogx(frequency_array,np.angle(fw),label='y = '+str(y))

        plt.xlabel('Frequency')
        plt.ylabel('arg(F(w))')
        plt.grid()
        plt.xlim(1,3e3)
plt.suptitle('PL: Mishra et al Fig 1')
plt.tight_layout()
plt.show()

#### Fig 2 : WIP, needs to be fixed.

In [ ]:
gamma=-0.8
Psi_shear=lenses.Psi_Ext(p_phys={'gamma1':gamma*np.cos(np.pi/8),'gamma2':-gamma*np.sin(np.pi/8)})
y = 0.8 
print('mu exp.: ' , 1/(1-gamma**2))
Cprec = {'no_warnings':1, 'no_errors':1, 'no_gslerrors':1}
#Cprec = {'no_output':1}

p_prec_t={'C_prec':Cprec,'eval_mode':'exact'}
I1=time_domain_c.It_MultiContour_C(Psi_shear, y, p_prec=p_prec_t)
I1.display_images()

x1,x2=I1.p_crits[0]['x1'],I1.p_crits[0]['x2']
fig, ax = tools.plot_potential(y, Psi_shear, levels=25)
ax.set_title('gamma = ' + str(gamma) )
ax.plot(x1,x2,'x')
ax.plot(y,0,'x')
fig.show()

In [ ]:
#x1,x2=-3.391036e-01, 3.061467e-01
eps=1e-3
Psi_PM_off=lenses.Psi_offcenterPointLens(p_phys={'xc1':x1+eps,'xc2':x2+eps })
Psi = lenses.CombinedLens({'lenses' : [Psi_PM_off, Psi_shear]})
I=time_domain_c.It_MultiContour_C(Psi, y)
fig, ax = tools.plot_potential(y, Psi, levels=25)
ax.plot(x1,x2,'x')
ax.plot(y,0,'x')

I.display_images()

In [ ]:
frequency_array = np.linspace(0,3e3,10000)
MLz=100*1.5
p_prec_t.update({'tmax':1e1, 'tmin':1e-5,'Nt':int(1e5)})

Fw = freq_domain_c.Fw_FFT_C(I)
ws=GMsun8pi*MLz*frequency_array
Fws= Fw.eval_Fw(ws)
plt.figure(1,figsize= (10,3))
plt.subplot(131)

plt.plot(I.t_grid, I.It_grid/2/np.pi,label='gamma = ' + str(gamma))
plt.xlabel(r'$\tau$')
plt.ylabel(r'$I(\tau)/2\pi$')
plt.axhline(1/(1-gamma**2)**0.5,c='k',ls='dashed')

plt.xscale('log')
#plt.xlim(1e-6,1e1)
#plt.ylim(0.9,4)
plt.legend()
plt.subplot(132)

plt.loglog(frequency_array,np.abs(Fws),label='gamma = ' + str(gamma))

plt.legend()
plt.xlabel('Frequency')
plt.ylabel('|F(w)|')
plt.grid()
#plt.xlim(1e1,1e3)
#plt.ylim(0.01,4)
plt.subplot(133)

plt.semilogx(frequency_array,np.angle(Fws),label='gamma = ' + str(gamma))

plt.xlabel('Frequency')
plt.ylabel('arg(F(w))')
plt.grid()
#plt.xlim(1e1,1e3)
#plt.ylim(-4,4)
plt.suptitle('PL shear minima: Mishra et al Fig 2')
plt.tight_layout()
plt.show()  

In [ ]:
frequency_array = np.linspace(0,3e3,10000)
MLz=100*1.5
plt.figure(1,figsize= (10,3))
eps= 1e-3
for gamma in [-0.2,-0.5,-0.8]:
    Psi_shear=lenses.Psi_Ext(p_phys={'gamma1':gamma*np.cos(np.pi/8),'gamma2':-gamma*np.sin(np.pi/8)})
    #Psi = lenses.CombinedLens({'lenses' : [Psi_PM, Psi_shear]})
    y = 0.4 
    Cprec = {'no_warnings':1, 'no_errors':1, 'no_gslerrors':1}
    #Cprec = {'no_output':1}

    p_prec_t={'C_prec':Cprec,'eval_mode':'exact'}.copy()
    #p_prec_t, p_prec_f = get_p_prec_optimal( MLz) 
    #p_prec_t.update({'tmax':1e1, 'tmin':1e-5,'Nt':int(1e5)})
    I1=time_domain_c.It_MultiContour_C(Psi_shear, y, p_prec=p_prec_t)
    #I1.display_images()

    x1,x2=I1.p_crits[0]['x1'],I1.p_crits[0]['x2']
    fig, ax = tools.plot_potential(y, Psi_shear, levels=25)
    ax.set_title('gamma = ' + str(gamma) )
    ax.plot(x1,x2,'x')
    ax.plot(y,0,'x')
    fig.show()
    Psi_PM_off=lenses.Psi_offcenterPointLens(p_phys={'xc1':x1+eps,'xc2':x2+eps })
    Psi = lenses.CombinedLens({'lenses' : [Psi_PM_off, Psi_shear]})
    print('mu exp.: ' , 1/(1-gamma**2))
    fig, ax = tools.plot_potential(y, Psi, levels=25)
    ax.plot(x1,x2,'x')
    ax.plot(y,0,'x')
    
    ax.set_title('PL min + gamma = ' + str(gamma) )
    fig.show()
    
    p_prec_t.update({'tmax':1e1, 'tmin':1e-5,'Nt':int(1e5)})
    I=time_domain_c.It_MultiContour_C(Psi, y)
    #I.display_images()
    Fw = freq_domain_c.Fw_FFT_C(I)
    ws=GMsun8pi*MLz*frequency_array
    Fws= Fw.eval_Fw(ws)
    plt.figure(1)
    plt.subplot(131)

    plt.plot(I.t_grid, I.It_grid/2/np.pi,label='gamma = ' + str(gamma))
    plt.xlabel(r'$\tau$')
    plt.ylabel(r'$I(\tau)/2\pi$')
    plt.axhline(1/(1-gamma**2)**0.5,c='k',ls='dashed')

    plt.xscale('log')
    #plt.xlim(1e-6,1e1)
    #plt.ylim(0.9,4)
    plt.legend()
    plt.subplot(132)

    plt.loglog(frequency_array,np.abs(Fws),label='gamma = ' + str(gamma))

    plt.legend()
    plt.xlabel('Frequency')
    plt.ylabel('|F(w)|')
    plt.grid()
    #plt.xlim(1e1,1e3)
    #plt.ylim(0.01,4)
    plt.subplot(133)

    plt.semilogx(frequency_array,np.angle(Fws),label='gamma = ' + str(gamma))

    plt.xlabel('Frequency')
    plt.ylabel('arg(F(w))')
    plt.grid()
    #plt.xlim(1e1,1e3)
    #plt.ylim(-4,4)
plt.suptitle('PL shear minima: Mishra et al Fig 2')
plt.tight_layout()
plt.show()  